In [55]:
%matplotlib
from biorefineries import cane as c
import biosteam as bst
from biosteam.plots import sankey
from biosteam.utils import colors, CABBI_colors
from colorpalette import ColorWheel, Color
from matplotlib import cm
import numpy as np
bst.nbtutorial()

c.YRCP2023()
color = CABBI_colors.grey.tint(20)
#loaded = set()


class CarbonHandle(sankey.Handle):
    __slots__ = ('node_colors', 'feedstock')
    
    def _init(self, node_colors, feedstock):
        self.node_colors = node_colors
        self.feedstock = feedstock
    
    def process_color(self, units, index):
        color = self.node_colors[index]
        RGB = color.RGB
        return "rgba(%d, %d, %d, 1.0)" %tuple(RGB)
    
    def stream_width(self, stream): 
        return self.get_carbon(stream)
    
    def stream_color(self, stream): 
        return "rgba(%d, %d, %d, 1.0)" %tuple(color.RGB)
    
    def get_carbon(self, stream):
        value = stream.get_atomic_flow('C') / self.feedstock.get_atomic_flow('C')
        if value < 2e-2: value = 0.
        return value
    
@bst.utils.piping.ignore_docking_warnings
def create_sankey_plot(name, height=300):
    try:
        brf = c.Biorefinery(name)
        brf.sys.simulate()
        if brf.configuration.number == 7:
            brf.vinasse.ID = 'wastewater'
            brf.wastewater_2.ID = 'wastewater'
            brf.filter_cake.ID = 'bagasse'
            brf.fiber_fines.ID = 'bagasse'
        elif brf.configuration.number == 9:
            brf.s241.ID = 'wastewater'
            brf.pretreatment_wastewater.ID = 'wastewater'
            brf.filter_cake.ID = 'bagasse'
            brf.fiber_fines.ID = 'bagasse'
        *sankey_groups, utilities, hxn, chp, wwt, storage = brf.unit_groups
        for i in storage.units: 
            if 'storage' in i.line.lower(): i.replace_with()
        area_colors = c.area_colors_biodiesel
        handle = CarbonHandle(
            max_feeds=100, max_products=100,
            node_colors=[area_colors[i.name] for i in sankey_groups],
            feedstock=brf.feedstock,
        )
        dct = globals()
        dct.update(locals())
        nodes = handle.nodes(sankey_groups)
        links = sum([i.links() for i in nodes], [])
        data = sankey.sankey_data(nodes, links=links, orientation="v", add_label=False)
        fig = sankey.sankey_figure(data)
        fig.update_layout(font_size=10, width=280, height=height)
        fig.show()
    finally:
        dct = globals()
        dct.update(locals())

Using matplotlib backend: QtAgg


In [56]:
create_sankey_plot('O7.1566', height=500)

In [46]:
feeds = [i for i in links if isinstance(i.source, sankey.FeedNode)]
products = [i for i in links if isinstance(i.sink, sankey.ProductNode)]
total = sum([i.value() for i in feeds])
for i in products: print(i, f"{i.value()/ total:.1%}")
print(sum([i.value() for i in products]) / total)

<Link: Biodiesel production - biodiesel> 7.5%
<Link: Juicing - bagasse> 64.8%
<Link: Oil production - vent> 10.6%
<Link: Oil production - cellmass> 16.6%
0.9961770098209481


In [47]:
create_sankey_plot('O9.1566', height=500)

In [34]:
feeds = [i for i in links if isinstance(i.source, sankey.FeedNode)]
products = [i for i in links if isinstance(i.sink, sankey.ProductNode)]
total = sum([i.value() for i in feeds])
for i in products: print(i, f"{i.value()/ total:.1%}")
print(sum([i.value() for i in products]) / total)

<Link: Biodiesel production - biodiesel> 13.3%
<Link: Oil production - lignin> 29.6%
<Link: Oil production - yeast extract> 30.5%
<Link: Oil production - vent> 19.4%
<Link: Juicing - bagasse> 2.1%
<Link: Oil production - wastewater> 4.8%
0.9978291877197283


In [52]:
create_sankey_plot('O8.1566', height=500)

In [7]:
feeds = [i for i in links if isinstance(i.source, sankey.FeedNode)]
products = [i for i in links if isinstance(i.sink, sankey.ProductNode)]
total = sum([i.value() for i in feeds])
for i in products: print(i, f"{i.value()/ total:.1%}")
print(sum([i.value() for i in products]) / total)

<Link: Biodiesel production - biodiesel> 10.3%
<Link: Oil production - vent> 19.4%
<Link: Juicing - filter cake> 2.1%
<Link: Oil production - wastewater> 4.8%
<Link: Oil production - lignin> 63.2%
0.998273531933631
